## Energy Efficiency Indices for carbon foot printing and as instrument for CO2 emission reduction of inland vessels
Federal Ministry of Transport and Digital Infrastructure, Germany; DST; CESNI 

----------------------------

#### The inland EEDI approach includes the attained EEDI and required EEDI for inland waterway navigation, covering 4 representative vessel types and 3 waterway conditions: 
#### four representative vessel types
- dry cargo and container selfpropelled ships
- tankers
- pushed convoys
- passenger ships

#### three waterway conditions: 
- deep calm water (h>7.5m), 
- shallow water with currents (3.5 m <= h <= 7.5 m), 
- cannal: major canals with trapezoidal cross-section, e.g. the MD canal.

#### references
- https://www.cesni.eu/wp-content/uploads/2021/03/cesnipt_energyindex_de.pdf project report
- https://platina3.eu/download/gernot-paulii-and-jens-ley-on-energy-efficiency-indices-as-an-instrument-for-the-reduction-of-co2-emissions-of-inland-vessels/ Platina3 presentation
- https://link.springer.com/book/10.1007/978-3-030-77325-0 book chapter 6.2.4
- https://www.prominent-iwt.eu/wp-content/uploads/2015/06/2015_09_23-PROMINENT-D1.1-ANNEX-A3-List-of-representative-journeys.pdf RhineARA dataset

In [1]:
import math
import itertools
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import tqdm
import plotly.express as px
from plotly.subplots import make_subplots

### attained EEDI   (g CO2 / ton-km)
for different vessel types and sailing conditions, use the same EEDI_attained formula with different P_D and V_g  to get the attained EEDI. 

Note that the P_D calculations for pushed convoys and self-propelled ships are the same in shallow water!! 
then the use the same formula for EEDI_attained, which means in shallow water the method has no difference among vessel types for getting EEDI_attained


In [2]:
def calculate_attained_EEDI( B, h, vessel_type, DWT, V_g):
    '''
    calculate_attained_EEDI for inland vessels.
    vessel types include 1) dry cargo and container selfpropelled ships, 2) tankers, 3) pushed convoys and 4) passenger ships.
   
    Note 
    - the calculation covers waterway conditions of deep calm water and shallow current water, vessel types of dry cargo and container selfpropelled ship, and pushed convoy so far, 
    for the RhineARA case study. Other water conditions and vessel types can be added into calculation when needed.
    
    input
    - B: ship beam (m)
    - h:  water depth, deep water range h >7.5 m, shallow water 3.5 m <= h <= 7.5 m;  lower than 3.5 m is excluded in the approach
    - vessel_type: "dry cargo", "container", "pushed convoy"   
    - DWT: capacity, [ton]
    - V_g: Velocity over ground, [km/h]
    - CF: CO2 Factor of gasoil, [g CO2/ g diesel] 
    - SFC: Specific fuel consumption, [g diesel/kWh] 
    
    '''
    def get_P_D( B, h, vessel_type, DWT):
        ''' P_D: Applied delivered power depending on ship type and sailing direction up-downstream
        '''
        if h > 7.5: 
            if vessel_type == "dry cargo" or vessel_type == "container":
                P_D = 0.262 * DWT        
            elif vessel_type == "pushed convoy":
                P_D = (0.146 + 0.25 * math.exp( B / (-11))) * DWT


        elif 3.5 <= h <= 7.5:
            if vessel_type == "dry cargo" or vessel_type == "container":
                P_D = (0.375 + 0.0625 * math.exp(-0.13 * B) - 0.5 * math.exp( h / (-2.8))) * DWT        
            elif vessel_type == "pushed convoy":
                P_D = (0.375 + 0.0625 * math.exp(-0.13 * B) - 0.5 * math.exp( h / (-2.8))) * DWT  
                # note that the P_D calculations for pushed convoys and self-propelled ships are the same in shallow water!! 
                # then the use the same formula for EEDI_attained, which means in shallow water the method has no difference among vessel types for getting EEDI_attained

        else:
            print ("out of the water depth range or wrong vessel type input")        

        return P_D
    
    
    CF = 3.206 
    SFC = 220 
    
    EEDI_attained = CF * SFC * get_P_D(B, h, vessel_type, DWT) / (DWT * V_g)
    
    return EEDI_attained

In [3]:
def get_P_D( B, h, vessel_type, DWT):
        ''' P_D: Applied delivered power depending on ship type and sailing direction up-downstream
        '''
        if h > 7.5: 
            if vessel_type == "dry cargo" or vessel_type == "container":
                P_D = 0.262 * DWT        
            elif vessel_type == "pushed convoy":
                P_D = (0.146 + 0.25 * math.exp( B / (-11))) * DWT


        elif 3.5 <= h <= 7.5:
            if vessel_type == "dry cargo" or vessel_type == "container":
                P_D = (0.375 + 0.0625 * math.exp(-0.13 * B) - 0.5 * math.exp( h / (-2.8))) * DWT        
            elif vessel_type == "pushed convoy":
                P_D = (0.375 + 0.0625 * math.exp(-0.13 * B) - 0.5 * math.exp( h / (-2.8))) * DWT  
                # note that the P_D calculations for pushed convoys and self-propelled ships are the same in shallow water!! 
                # then the use the same formula for EEDI_attained, which means in shallow water the method has no difference among vessel types for getting EEDI_attained

        else:
            print ("out of the water depth range or wrong vessel type input")        

        return P_D

In [4]:
EEDI_attained = calculate_attained_EEDI( B = 10, h =4, vessel_type = "container", DWT =3000, V_g=11)
EEDI_attained

17.45395893226047

In [5]:
EEDI_attained = calculate_attained_EEDI( B = 10, h =10, vessel_type = "container", DWT =3000, V_g=11)
EEDI_attained

16.799439999999997

In [6]:
EEDI_attained = calculate_attained_EEDI( B = 20, h =10, vessel_type = "pushed convoy", DWT =18000, V_g=13)
EEDI_attained

10.12297795151502

In [7]:
EEDI_attained = calculate_attained_EEDI( B = 11.4, h =10, vessel_type = "container", DWT =5520, V_g=18)
EEDI_attained

10.266324444444445

### required EEDI (g CO2 / ton-km)
for different vessel types and sailing conditions, use different EEDI_req formula below to get the required EEDI for each situation

Note that the required_EEDI calculations for pushed convoys and self-propelled ships are the same in deep calm water!!

In [8]:
def calculate_required_EEDI(h, U_c, vessel_type, DWT):
    ''' the calculation covers waterway conditions of deep calm water and shallow current water, vessel types of dry cargo and container selfpropelled ship, and pushed convoy so far, 
    for the RhineARA case study. Other water conditions and vessel types can be added into calculation when needed.
    
    - U_c: current speed, [km/h]
    '''
    if h > 7.5: 
        if vessel_type == "dry cargo" or vessel_type == "container":
            EEDI_req = 10 + 13 * math.exp( DWT / (-470)) + 8 * math.exp( DWT / (-4500))      
        elif vessel_type == "pushed convoy":
            EEDI_req = 10 + 13 * math.exp( DWT / (-470)) + 8 * math.exp( DWT / (-4500))
        # note that the required_EEDI calculations for pushed convoys and self-propelled ships are the same in deep calm water!!

    elif 3.5 <= h <= 7.5:
        if vessel_type == "dry cargo" or vessel_type == "container":
            EEDI_req = (21 + 0.7 * U_c + 0.28 * U_c**2) + (11 + 0.78 * U_c - 0.46 * U_c**2 + 0.154 * U_c**3)* math.exp(DWT / (-800))       
        elif vessel_type == "pushed convoy":
            EEDI_req = (18 - 2.5 * U_c + 0.75 * U_c**2) + (8 + 0.25 * U_c + 0.375 * U_c**2) * math.exp(DWT / (-3100))
    else:
            print ("out of the water depth range or wrong vessel type input")        
        
    
    return EEDI_req

In [9]:
EEDI_req = calculate_required_EEDI(h = 4, U_c = 2,  vessel_type = "pushed convoy", DWT =1000)
EEDI_req

23.24277519974214

In [10]:
EEDI_req = calculate_required_EEDI(h = 4, U_c = 2,  vessel_type = "container", DWT =250)
EEDI_req

32.26426999717026

In [11]:
EEDI_req = calculate_required_EEDI(h = 10, U_c = 4,  vessel_type = "container", DWT =5520)
EEDI_req

12.34625120362715

In [12]:
# to do input table

### Application for RhineARA corrior
take U_c = 4 km/h as the general current speed, take h = 10 m representing deep water and h = 4 m representing shallow water

####  input value

In [13]:
h = [4, 10]  
U_c = [4]
vessel_type = ["dry cargo","container","pushed convoy"]
DWT = [11200, 5520, 5320, 5500, 5686, 3043, 5920, 6228, 3300, 3043, 8134, 2973, 1680, 3257, 3043, 2211, 2075, 2908, 985,1522,1718 ]
B = [15, 11.4, 11.45, 17, 14.2, 9.5, 8.2]
V_g = [12.4, 10, 10.8, 10.1, 9.9, 8.9, 9.5, 11.4, 10.3, 13.2, 10.4, 9.6, 10.5,18]
journey = ["Rotterdam_Duisburg_PushB4",
            "Rotterdam_Antwerp_C3LB",
            "Rotterdam_Karlsruhe_MTS135",
            "Amsterdam_Karlsruhe_C3LB",
            "Rotterdam_Basel_C3LB",
            "Antwerp_Thionvile_MVS110",
            "Amsterdam_Antwerp_C3LB",
            "Rotterdam_Krotzenburg_C3LB",
            "Amsterdam_Rotterdam_MTS135",
            "Antwerp_Mainz_MVS135",
            "Breisach_Cuijk_MVS110",
            "Antwerp_Duisburg_C3LB",
            "Rotterdam_Duisburg_MVS110",
            "Rotterdam_Ludwigshafen_MTS86",
            "Rotterdam_Kampen_MTS110",
            "Rotterdam_Strassbourg_MVS110",
            "Amsterdam_Heilbronn_MVS105",
            "Duisburg_Antwerp_MVS110",
            "Rotterdam_Alphen_MVS105",
            "Terneuzen_Rotterdam_MTS110",
            "Wesel_Enkhuizen_MVS67",
            "Rotterdam_Herne_MVS86",
            "Dusseldorf_Antwerp_MVS110",
            "Antwerp_Gent_MVS110",
            "Rotterdam_Duisburg_MVS86"
            ]

#### prepare input matrix for calculation

In [14]:
# prepare the work to be done
# create a list of all combinations
work = list(itertools.product(journey, vessel_type, h, U_c, DWT, B, V_g))

# prepare a list of dictionaries for pandas
rows = []
for item in work:
    row = {"journey": item[0],"vessel_type": item[1], "h": item[2], "U_c": item[3], "DWT": item[4], "B": item[5], "V_g": item[6]}
    rows.append(row)

# these are all the simulations that we want to run
# convert them to dataframe, so that we can apply a function and monitor progress
work_df = pd.DataFrame(rows)
work_df.head()

,journey,vessel_type,h,U_c,DWT,B,V_g
0,Rotterdam_Duisburg_PushB4,dry cargo,4,4,11200,15.0,12.4
1,Rotterdam_Duisburg_PushB4,dry cargo,4,4,11200,15.0,10.0
2,Rotterdam_Duisburg_PushB4,dry cargo,4,4,11200,15.0,10.8
3,Rotterdam_Duisburg_PushB4,dry cargo,4,4,11200,15.0,10.1
4,Rotterdam_Duisburg_PushB4,dry cargo,4,4,11200,15.0,9.9


In [15]:

results = []

for i, row in tqdm.tqdm(work_df.iterrows()):
    # create a new vessel, like the one above (so that it also has L)
    journey = row['journey']
    vessel_type = row['vessel_type']
    h = row['h']
    U_c = row['U_c']
    DWT = row['DWT']
    B = row['B']    
    V_g = row['V_g']


    P_D = get_P_D( B, h, vessel_type, DWT)
    EEDI_attained = calculate_attained_EEDI( B, h, vessel_type, DWT, V_g)
    EEDI_req = calculate_required_EEDI(h, U_c, vessel_type, DWT)
    if EEDI_attained <= EEDI_req:
        meet_req =  "Yes"
    else:
        meet_req =  "No"
        
    result = {}
    result.update(row)
    result['P_D'] = P_D
    result['EEDI_req (g CO2 / ton-km)'] = EEDI_req    
    result['EEDI_attained (g CO2 / ton-km)'] = EEDI_attained
    result['meet_req'] = meet_req

    results.append(result)

308700it [01:03, 4857.37it/s]


In [16]:
journey_df = pd.DataFrame(results)

journey_df.head()

,journey,vessel_type,h,U_c,DWT,B,V_g,P_D,EEDI_req (g CO2 / ton-km),EEDI_attained (g CO2 / ton-km),meet_req
0,Rotterdam_Duisburg_PushB4,dry cargo,4,4,11200,15.0,12.4,2957.546046,28.280014,15.020279,Yes
1,Rotterdam_Duisburg_PushB4,dry cargo,4,4,11200,15.0,10.0,2957.546046,28.280014,18.625146,Yes
2,Rotterdam_Duisburg_PushB4,dry cargo,4,4,11200,15.0,10.8,2957.546046,28.280014,17.245506,Yes
3,Rotterdam_Duisburg_PushB4,dry cargo,4,4,11200,15.0,10.1,2957.546046,28.280014,18.440739,Yes
4,Rotterdam_Duisburg_PushB4,dry cargo,4,4,11200,15.0,9.9,2957.546046,28.280014,18.813279,Yes


In [17]:
# 25 representative journeys in RhineARA corridor
j1= journey_df.query('journey == "Rotterdam_Duisburg_PushB4" & vessel_type =="pushed convoy" & DWT == 11200 & h == 4 & B == 15  & V_g == 12.4')
j2= journey_df.query('journey == "Rotterdam_Antwerp_C3LB" & vessel_type =="container" & DWT == 5520 & h == 10 & B == 11.4  & V_g == 18') # coupled convoy is excluded in EEDI Inland approach
j3= journey_df.query('journey == "Rotterdam_Karlsruhe_MTS135" & vessel_type =="dry cargo" & DWT == 5320 & h == 4 & B == 11.45  & V_g == 10.8') # liquid bulk in actual 
j4= journey_df.query('journey == "Amsterdam_Karlsruhe_C3LB" & vessel_type =="dry cargo" & DWT == 5500 & h == 4 & B == 11.4  & V_g == 10.1')
j5= journey_df.query('journey == "Rotterdam_Basel_C3LB" & vessel_type =="container" & DWT == 5686 & h == 4 & B == 11.45  & V_g == 9.9')
j6= journey_df.query('journey == "Antwerp_Thionvile_MVS110" & vessel_type =="dry cargo" & DWT == 3043 & h == 4 & B == 11.4  & V_g == 8.9')
j7= journey_df.query('journey == "Amsterdam_Antwerp_C3LB" & vessel_type =="container" & DWT == 5520 & h == 4 & B == 11.4  & V_g == 10.8')
j8= journey_df.query('journey == "Rotterdam_Krotzenburg_C3LB" & vessel_type =="dry cargo" & DWT == 5920 & h == 4 & B == 11.45  & V_g == 9.5')
j9= journey_df.query('journey == "Amsterdam_Rotterdam_MTS135" & vessel_type =="dry cargo" & DWT == 6228 & h == 4 & B == 17  & V_g == 11.4') # liquid bulk in actual 
j10= journey_df.query('journey == "Antwerp_Mainz_MVS135" & vessel_type =="container" & DWT == 3300 & h == 4 & B == 11.45  & V_g == 10.3')
j11= journey_df.query('journey == "Breisach_Cuijk_MVS110" & vessel_type =="dry cargo" & DWT == 3043 & h == 4 & B == 11.45  & V_g == 9.9') # downstream
j12= journey_df.query('journey == "Antwerp_Duisburg_C3LB" & vessel_type =="container" & DWT == 8134 & h == 4 & B == 14.2  & V_g == 10.3')
j13= journey_df.query('journey == "Rotterdam_Duisburg_MVS110" & vessel_type =="container" & DWT == 2973 & h == 4 & B == 11.4  & V_g == 10.4')
j14= journey_df.query('journey == "Rotterdam_Ludwigshafen_MTS86" & vessel_type =="container" & DWT == 1680 & h == 4 & B == 9.5  & V_g == 10.3') # liquid bulk in actual 
j15= journey_df.query('journey == "Rotterdam_Kampen_MTS110" & vessel_type =="container" & DWT == 3257 & h == 4 & B == 11.4  & V_g == 13.2') # liquid bulk in actual 
j16= journey_df.query('journey == "Rotterdam_Strassbourg_MVS110" & vessel_type =="dry cargo" & DWT == 3043 & h == 4 & B == 11.4  & V_g == 10.1')
j17= journey_df.query('journey == "Amsterdam_Heilbronn_MVS105" & vessel_type =="dry cargo" & DWT == 2211 & h == 4 & B == 11.4  & V_g == 9.6')
j18= journey_df.query('journey == "Duisburg_Antwerp_MVS110" & vessel_type =="dry cargo" & DWT == 3043 & h == 4 & B == 11.4  & V_g == 10.4')  # downstream
j19= journey_df.query('journey == "Rotterdam_Alphen_MVS105" & vessel_type =="container" & DWT == 2075 & h == 4 & B == 11.4  & V_g == 9.5') 
j20= journey_df.query('journey == "Terneuzen_Rotterdam_MTS110" & vessel_type =="container" & DWT == 2908 & h == 10 & B == 11.4  & V_g == 18') # liquid bulk in actual
j21= journey_df.query('journey == "Wesel_Enkhuizen_MVS67" & vessel_type =="dry cargo" & DWT == 985 & h == 4 & B == 8.2  & V_g == 10.1') # downstream
j22= journey_df.query('journey == "Rotterdam_Herne_MVS86" & vessel_type =="dry cargo" & DWT == 1522 & h == 4 & B == 9.5  & V_g == 9.6') 
j23= journey_df.query('journey == "Dusseldorf_Antwerp_MVS110" & vessel_type =="dry cargo" & DWT == 3043 & h == 4 & B == 11.4  & V_g == 10.5') # downstream
j24= journey_df.query('journey == "Antwerp_Gent_MVS110" & vessel_type =="dry cargo" & DWT == 3043 & h == 10 & B == 11.4  & V_g == 18') 
j25= journey_df.query('journey == "Rotterdam_Duisburg_MVS86" & vessel_type =="dry cargo" & DWT == 1718 & h == 4 & B == 9.5  & V_g == 10.5') 




### Inland EEDI application results for RhineARA corrior

In [18]:

journeys_df = pd.concat([j1,j2,j3,j3,j4,j5,j6,j7,j8,j9,j10,j11,j12,j13,j14,j15,j16,j17,j18,j19,j20,j21,j22,j23,j24,j25])

In [19]:
journeys = journeys_df.drop_duplicates()
journeys

,journey,vessel_type,h,U_c,DWT,B,V_g,P_D,EEDI_req (g CO2 / ton-km),EEDI_attained (g CO2 / ton-km),meet_req
8232,Rotterdam_Duisburg_PushB4,pushed convoy,4,4,11200,15.00,12.4,2957.546046,20.404601,15.020279,Yes
18647,Rotterdam_Antwerp_C3LB,container,10,4,5520,11.40,18.0,1446.240000,12.346251,10.266324,Yes
24922,Rotterdam_Karlsruhe_MTS135,dry cargo,4,4,5320,11.45,10.8,1432.577140,28.301501,17.586071,Yes
37355,Amsterdam_Karlsruhe_C3LB,dry cargo,4,4,5500,11.40,10.1,1481.553761,28.297169,18.811332,Yes
53932,Rotterdam_Basel_C3LB,container,4,4,5686,11.45,9.9,1531.134139,28.293608,19.184805,Yes
62249,Antwerp_Thionvile_MVS110,dry cargo,4,4,3043,11.40,8.9,819.703290,28.650321,21.347691,Yes
78318,Amsterdam_Antwerp_C3LB,container,4,4,5520,11.40,10.8,1486.941229,28.296745,17.592079,Yes
87058,Rotterdam_Krotzenburg_C3LB,dry cargo,4,4,5920,11.45,9.5,1594.145990,28.290157,19.992586,Yes
99519,Amsterdam_Rotterdam_MTS135,dry cargo,4,4,6228,17.00,11.4,1631.927650,28.286911,16.211890,Yes
116068,Antwerp_Mainz_MVS135,container,4,4,3300,11.45,10.3,888.628677,28.548573,18.439764,Yes


### Explanation of the inland EEDI application for the RhineARA corridor:

- Vessel types.  The greek symbols used in the formulas for the power, attained EEDI, required EEDI calculation in the original DST project report are made anonymous, thanks to the description of the approach in the book "Design of Contemporary Inland Waterway Vessels-The Case of the Danube River", some of greek symbols are unveiled to use, but only for dry cargo and container self-propelled vessels and pushed convoys, exclude the tankers and passanger ships. Therefore, the approach applied in this notebook only for dry cargo and container self-propelled vessels and pushed convoys. The journeys with tankers and coupled convoys in RhineARA are calcuted by self-propelled motor vessels as replacements.

- Waterway situations. The waterway situations only include deep water and shallow water in RhineARA, without considering the restricted canal.

- Vessel velocity. the attained EEDI calculation needs actual velocity measurement while sailing with the given P_D, the given P_D needs to be calculated in advance by knowing (B, h, vessel_type, DWT). Some of the velocity data log (deep waterjourneys 2,20,24) in Prominent project are not suitable to be directly used for getting attained EEDI. These velocity are estimated based on the calculated P_D to be used for EEDI_attained.

- Current velocity. The current is assumed as an average value of 4 km/h for all shallow situation in the required EEDI calculation.

- the given P_D is a function of ( B, vessel_type, DWT), in deep water, P_D calculation varys among vessel types; in shallow water, P_D calculation is the same for all cargo vessels if the unveiled greek symbols in the book are correct. 

- the attained EEDI is a function of ( B, vessel_type, DWT, measured vessel velocity with the given P_D) in deep and shallow water. 

- the required EEDI is a function of (vessel_type, DWT) for deep water, and (vessel_type, DWT, U_c) for shallow water.

- The inland EEDI application for the RhineARA shows that all the vessel-journeys in RhineARA meet the required EEDI.

### Critical comments on the inland EEDI approach:

- The proposed inland EEDI take water depth and current influence into consideration in a general way, which may be unable to deal with the actual IWT situation that the water depth and current vary frequently even along one IWT trip.  

- For the current, the approach includes only the sailing upstream situation, since the formula derived for required EEDI with current influence is only based on upstream current measurement.

- For the waterway depth consideration, the approach includes "deep calm water" and "shallow water with current". All the water depths > 7.5 m are defined as "deep calm water" and current is not considered in this situation. All the water depths between 3.5 m and 7.5 m are defined as shallow water. The specific water depth h is not used in the formula, but serve as a condition for choosing calculation formulas between deep and shallow water. However, the power and emissions results of the same vessel sailing at 7.5 m and 3.5 m are quite different in fact. This reflects the proposed inland EEDI is unable to fairly deal with the different water depth influence inside the 3.5~7.5 m range. 

- The water depth below 3.5 m is excluded in this approach, which may apply for the Rhine, but not for the Danube, since the Danube water depths are smaller with maintaince standard around 2.5 m.

